In [1]:
#!pip install git+https://github.com/nockchun/rspy --force
import rspy as rsp
rsp.setSystemWarning(off=True)

In [2]:
import os
from glob import glob
import pandas as pd

# 손익계산서 가져오기

In [3]:
DATA_YEAR = "2020"
DATA_QUARTER = "1Q"

In [4]:
dfs = []
for file in glob(f"data/{DATA_YEAR}_{DATA_QUARTER}_PL_*/*"):
    df = pd.read_csv(file, sep='\t', encoding='cp949')
    df = df.rename(columns={
        "당기 1분기 3개월" : "당기",
        "당기 반기 3개월" : "당기",
        "당기 3분기 3개월" : "당기"
    })
    dfs.append(df[["재무제표종류", "종목코드", "항목명", "당기"]])

In [5]:
pd.concat([dfs[0].head(1), dfs[1].head(1), dfs[2].head(1), dfs[3].head(1)])

,재무제표종류,종목코드,항목명,당기
0,"포괄손익계산서, 기능별 분류(세후기타포괄손익) - 별도재무제표",[095570],영업수익,"107,018,343,206"
0,"포괄손익계산서, 기능별 분류(세후기타포괄손익) - 연결재무제표",[095570],영업수익,"316,655,908,554"
0,"손익계산서, 기능별 분류 - 별도재무제표",[001040],영업수익,"69,748,436,000"
0,"손익계산서, 기능별 분류 - 연결재무제표",[001040],수익(매출액),"7,840,010,949,000"


# 데이터 전처리

In [6]:
for df in dfs:
    df["보고년도"] = DATA_YEAR
    df["보고분기"] = DATA_QUARTER
    df["연결유무"] = False
    df["포괄유무"] = False
    df.당기 = df.당기.str.replace(",", "").astype(float)
    df.fillna(0, inplace=True)    

In [7]:
def fixData(row):
    try:
        row.연결유무 = "연결재무제표" in row.재무제표종류
        row.포괄유무  = "포괄손익계산서" in row.재무제표종류
        row.종목코드 = row.종목코드[1:-1]
        row.보고분기 = DATA_QUARTER
        row.항목명 = str(row.항목명).strip()
    except:
        print(row)
    return row;

In [8]:
dfs_final = []
for df in dfs:
    df = df.apply(fixData, axis=1)
    df = df[[
        "종목코드", "연결유무", "포괄유무", "보고년도", "보고분기", "항목명", "당기"
    ]]
    df = df.astype({
        "종목코드":"str", "연결유무":"boolean", "포괄유무":"boolean", "보고년도":"str", "보고분기":"str", "항목명":"str", "당기":"int64"
    })
    dfs_final.append(df)

# DB에 저장.

In [9]:
rows = []
for df in dfs_final:
    for index, row in df.iterrows():
        rows.append(tuple(row))

In [10]:
rows[:5]

[('095570', False, True, '2020', '1Q', '영업수익', 107018343206),
 ('095570', False, True, '2020', '1Q', '영업비용', 96020522594),
 ('095570', False, True, '2020', '1Q', '영업이익(손실)', 10997820612),
 ('095570', False, True, '2020', '1Q', '기타수익', 1043783837),
 ('095570', False, True, '2020', '1Q', '기타비용', 2803423891)]

In [11]:
db = rsp.PGManager()

In [12]:
db.createPool("postgres", "postgres", "postgres", "192.168.0.200")
db.setMapper("mapper.xml")

In [13]:
db.executeMany("inStateProfitLoss", rows)